**Подзапросы**

Используя данные из таблицы user_actions, рассчитайте среднее число заказов всех пользователей нашего сервиса.
Для этого сначала в подзапросе посчитайте, сколько заказов сделал каждый пользователь, а затем обратитесь к результату подзапроса в блоке FROM и уже в основном запросе усредните количество заказов по всем пользователям.
Полученное среднее число заказов всех пользователей округлите до двух знаков после запятой. Колонку с этим значением назовите orders_avg.

In [ ]:
SELECT 
	ROUND(AVG(orders_count), 2) AS orders_avg
FROM
	(SELECT 
	 	COUNT(DISTINCT(order_id)) AS orders_count
     FROM
	 	user_actions
     GROUP BY 
	 	user_id
	) as subquery

Повторите запрос из предыдущего задания, но теперь вместо подзапроса используйте оператор WITH и табличное выражение.
Условия задачи те же: используя данные из таблицы user_actions, рассчитайте среднее число заказов всех пользователей.
Полученное среднее число заказов округлите до двух знаков после запятой. Колонку с этим значением назовите orders_avg.

In [ ]:
WITH 
subquery AS (
	SELECT 
		COUNT(DISTINCT(order_id)) AS order_count
    FROM   
		user_actions
    GROUP BY 
		user_id
)
SELECT 
	ROUND(AVG(order_count), 2) AS orders_avg
FROM
   subquery

Выведите из таблицы products информацию о всех товарах кроме самого дешёвого.
Результат отсортируйте по убыванию id товара.

In [ ]:
SELECT 
	product_id,
	name,
	price
FROM   
	products
WHERE  
	price > (SELECT MIN(price) FROM products)
ORDER BY 
	product_id DESC

Выведите информацию о товарах в таблице products, цена на которые превышает среднюю цену всех товаров на 20 рублей и более. Результат отсортируйте по убыванию id товара.

In [ ]:
wITH 
subquery AS (
	SELECT 
		AVG(price) AS all_price
    FROM 
		products
)
SELECT 	
	product_id,
	name,
	price
FROM   
	products
WHERE  
	(price - (SELECT * FROM subquery)) > 19
ORDER BY
	product_id DESC

Посчитайте количество уникальных клиентов в таблице user_actions, сделавших за последнюю неделю хотя бы один заказ.
Полученную колонку с числом клиентов назовите users_count. В качестве текущей даты, от которой откладывать неделю, используйте последнюю дату в той же таблице user_actions.

In [ ]:
wITH 
subquery AS (
	SELECT 
		MAX(time) AS max_time
    FROM
		user_actions
)
SELECT 
	COUNT(DISTINCT(user_id)) AS users_count
FROM   
	user_actions
WHERE  
	time > ((SELECT max_time FROM subquery) - interval '1 week') 
HAVING
	COUNT(order_id) > 0

С помощью функции AGE и агрегирующей функции снова определите возраст самого молодого курьера мужского пола в таблице couriers, но в этот раз при расчётах в качестве первой даты используйте последнюю дату из таблицы courier_actions.
Чтобы получить именно дату, перед применением функции AGE переведите последнюю дату из таблицы courier_actions в формат DATE, как мы делали в этом задании.
Возраст курьера измерьте количеством лет, месяцев и дней и переведите его в тип VARCHAR. Полученную колонку со значением возраста назовите min_age.

In [ ]:
wITH 
max_action_date AS (
	SELECT 
		MAX(time)::DATE AS max_date
    FROM
		courier_actions
)
SELECT 
	MIN(AGE(max_date, birth_date))::VARCHAR AS min_age
FROM   
	couriers, 
	max_action_date
WHERE  
	sex = 'male'

Из таблицы user_actions с помощью подзапроса или табличного выражения отберите все заказы, которые не были отменены пользователями.
Выведите колонку с id этих заказов. Результат запроса отсортируйте по возрастанию id заказа.
Добавьте в запрос оператор LIMIT и выведите только первые 1000 строк результирующей таблицы.

In [ ]:
wITH
subquery AS (
	SELECT 
		order_id
	FROM   
		user_actions
    WHERE  
		action = 'cancel_order'
)
SELECT 
	order_id
FROM   
	user_actions
WHERE  
	order_id NOT IN (SELECT order_id FROM   subquery)
ORDER BY 
	order_id ASC 
LIMIT 
	1000

Используя данные из таблицы user_actions, рассчитайте, сколько заказов сделал каждый пользователь и отразите это в столбце orders_count.
В отдельном столбце orders_avg напротив каждого пользователя укажите среднее число заказов всех пользователей, округлив его до двух знаков после запятой.
Также для каждого пользователя посчитайте отклонение числа заказов от среднего значения. Отклонение считайте так: число заказов «минус» округлённое среднее значение. Колонку с отклонением назовите orders_diff.
Результат отсортируйте по возрастанию id пользователя. Добавьте в запрос оператор LIMIT и выведите только первые 1000 строк результирующей таблицы.

In [ ]:
WITH 
user_orders AS (
	SELECT 
		user_id,
        COUNT(order_id) AS orders_count
    FROM 
		user_actions
    WHERE  
		action = 'create_order'
    GROUP BY 
		user_id
)
SELECT 
	user_id,
    COUNT(order_id) AS orders_count,
    ROUND((SELECT AVG(orders_count) FROM user_orders), 2) AS orders_avg, 
	COUNT(order_id) - ROUND((SELECT AVG(orders_count) FROM   user_orders), 2) AS orders_diff
FROM   
	user_actions
WHERE  
	action = 'create_order'
GROUP BY 
	user_id
ORDER BY 
	user_id ASC 
LIMIT 
	1000

Назначьте скидку 15% на товары, цена которых превышает среднюю цену на все товары на 50 и более рублей, а также скидку 10% на товары, цена которых ниже средней на 50 и более рублей. Цену остальных товаров внутри диапазона (среднее - 50; среднее + 50) оставьте без изменений. При расчёте средней цены, округлите её до двух знаков после запятой.
Выведите информацию о всех товарах с указанием старой и новой цены. Колонку с новой ценой назовите new_price.
Результат отсортируйте сначала по убыванию прежней цены в колонке price, затем по возрастанию id товара.

In [ ]:
WITH 
subquery AS (
	SELECT 
		ROUND(AVG(price), 2) AS avg_price
    FROM 
		products
)
SELECT 
	product_id,
    name,
    price,
    CASE 
		WHEN price >= (SELECT avg_price FROM subquery) + 50 THEN price * 0.85 
		WHEN price + 50 <= (SELECT avg_price FROM subquery) THEN price * 0.90 
		ELSE price 
	END AS new_price
FROM   
	products
ORDER BY 
	price DESC,
	product_id ASC

Выясните, есть ли в таблице courier_actions такие заказы, которые были приняты курьерами, но не были созданы пользователями. Посчитайте количество таких заказов.
Колонку с числом заказов назовите orders_count.

In [ ]:
WITH 
user_order_ids AS (
	SELECT 
		DISTINCT(order_id)
    FROM 
		user_actions
)
SELECT 
	COUNT(DISTINCT(order_id)) AS orders_count
FROM 
	courier_actions
WHERE 
	action = 'accept_order'
	AND order_id NOT IN (SELECT order_id FROM user_order_ids)

Выясните, есть ли в таблице courier_actions такие заказы, которые были приняты курьерами, но не были доставлены пользователям. Посчитайте количество таких заказов.
Колонку с числом заказов назовите orders_count.

In [ ]:
WITH 
subquery AS (
	SELECT 
		order_id
    FROM   
		user_actions
    WHERE  
		action = 'cancel_order'
)
SELECT 
	COUNT(order_id) AS orders_count
FROM   
	courier_actions
WHERE  
	action = 'accept_order'
	AND order_id IN (SELECT order_id FROM subquery)

Определите количество отменённых заказов в таблице courier_actions и выясните, есть ли в этой таблице такие заказы, которые были отменены пользователями, но при этом всё равно были доставлены. Посчитайте количество таких заказов.
Колонку с отменёнными заказами назовите orders_canceled. Колонку с отменёнными, но доставленными заказами назовите orders_canceled_and_delivered. 

In [ ]:
WITH
subquery AS (
		SELECT 
			DISTINCT(order_id)
        FROM   
			user_actions
        WHERE  
			action = 'cancel_order'
)
SELECT 
	COUNT(order_id) AS orders_canceled,
    COUNT(order_id) FILTER (WHERE action = 'deliver_orders') AS orders_canceled_and_delivered
FROM   
	courier_actions
WHERE  
	order_id IN (SELECT order_id FROM subquery)

По таблицам courier_actions и user_actions снова определите число недоставленных заказов и среди них посчитайте количество отменённых заказов и количество заказов, которые не были отменены (и соответственно, пока ещё не были доставлены).
Колонку с недоставленными заказами назовите orders_undelivered, колонку с отменёнными заказами назовите orders_canceled, колонку с заказами «в пути» назовите orders_in_process.

In [ ]:
WITH
subquery AS (
	SELECT 
		DISTINCT(order_id)
	FROM   
		user_actions
    WHERE  
		action = 'cancel_order'
)
SELECT 
	COUNT(DISTINCT(order_id)) AS orders_undelivered,
    COUNT(order_id) FILTER (WHERE action = 'accept_order') AS orders_canceled,
    COUNT(DISTINCT(order_id)) - COUNT(order_id) FILTER (WHERE action = 'accept_order') AS orders_in_process
FROM   
	courier_actions
WHERE  
	order_id IN (SELECT order_id FROM subquery)

Отберите из таблицы users пользователей мужского пола, которые старше всех пользователей женского пола.
Выведите две колонки: id пользователя и дату рождения. Результат отсортируйте по возрастанию id пользователя.

In [ ]:
WITH
female_birth_max AS(
	SELECT 
		MIN(birth_date) AS female_birth
    FROM   
		users
    WHERE  
		sex = 'female'
        AND birth_date is not null
)
SELECT 
	user_id,
    birth_date
FROM 
	users
WHERE  
	sex = 'male'
    AND birth_date < (SELECT female_birth FROM female_birth_max)
ORDER BY 
	user_id

Выведите id и содержимое 100 последних доставленных заказов из таблицы orders.
Содержимым заказов считаются списки с id входящих в заказ товаров. Результат отсортируйте по возрастанию id заказа.

In [ ]:
WITH 
subquery AS (
    SELECT order_id
    FROM
        courier_actions
    WHERE
        action = 'deliver_order'
    ORDER BY 
        time DESC
    LIMIT 
        100
)
SELECT 
    order_id,
    product_ids
FROM
    orders
WHERE 
    order_id IN (SELECT order_id FROM subquery)
ORDER BY 
    order_id

Из таблицы couriers выведите всю информацию о курьерах, которые в сентябре 2022 года доставили 30 и более заказов. Результат отсортируйте по возрастанию id курьера.

In [ ]:
WITH
subquery AS (
    SELECT    
        courier_id
    FROM 
        courier_actions
    WHERE
        action = 'deliver_order' 
        AND DATE_PART('month', time) = 9 
        AND DATE_PART('year', time) = 2022
    GROUP BY
        courier_id
    HAVING
        COUNT(DISTINCT(order_id)) > 29
)
SELECT *
FROM
    couriers
WHERE
    courier_id IN (SELECT courier_id FROM subquery) 
ORDER BY 
    courier_id

Рассчитайте средний размер заказов, отменённых пользователями мужского пола.
Средний размер заказа округлите до трёх знаков после запятой. Колонку со значением назовите avg_order_size.

In [ ]:
WITH
male_users AS (
    SELECT 
        user_id
    FROM
        users
    WHERE
        sex = 'male'
),
cancel_orders AS (
    SELECT
        order_id
    FROM 
        user_actions
    WHERE
        action = 'cancel_order'
        AND user_id IN (SELECT user_id FROM male_users)
)
SELECT
    ROUND(AVG(ARRAY_LENGTH(product_ids, 1)), 3) AS avg_order_size
FROM    
    orders
WHERE
    order_id IN (SELECT order_id FROM cancel_orders)

Посчитайте возраст каждого пользователя в таблице users.

In [ ]:
WITH 
date_max AS (
    SELECT
        MAX(time) as last_date
    FROM
        user_actions
), 
avg_age AS (
    SELECT
        AVG(DATE_PART('year', AGE((SELECT last_date FROM date_max),birth_date)))::INTEGER AS avg_val
    FROM
        users
    WHERE
        birth_date IS NOT NULL
)
SELECT
    user_id,
    CASE
    WHEN birth_date IS NULL THEN (SELECT avg_val FROM avg_age)
    ELSE DATE_PART('year', AGE((SELECT last_date FROM date_max), birth_date))::INTEGER
    END AS age
FROM 
    users
ORDER BY 
    user_id

Для каждого заказа, в котором больше 5 товаров, рассчитайте время, затраченное на его доставку. 
В результат включите id заказа, время принятия заказа курьером, время доставки заказа и время, затраченное на доставку. Новые колонки назовите соответственно time_accepted, time_delivered и delivery_time.
В расчётах учитывайте только неотменённые заказы. Время, затраченное на доставку, выразите в минутах, округлив значения до целого числа. Результат отсортируйте по возрастанию id заказа.

In [ ]:
WITH 
cancel_orders AS (
    SELECT 
        DISTINCT(order_id)
    FROM 
        user_actions
    WHERE
        action != 'cancel_orders'
),
count_products AS (
    SELECT
        DISTINCT(order_id)
    FROM 
        orders
    WHERE 
        order_id IN (SELECT order_id FROM cancel_orders)
        AND ARRAY_LENGTH(product_ids, 1) > 5
)
SELECT 
    DISTINCT(order_id),
    MIN(time) AS time_accepted,
    MAX(time) AS time_delivered,
    EXTRACT(epoch FROM (MAX(time) - MIN(time))/60)::INTEGER AS delivery_time
FROM
    courier_actions
WHERE
    order_id IN (SELECT order_id FROM count_products)
GROUP BY
    order_id
HAVING 
    (EXTRACT(epoch FROM (MAX(time) - MIN(time))/60)::INTEGER) > 0
ORDER BY 
    order_id

Для каждой даты в таблице user_actions посчитайте количество первых заказов, совершённых пользователями.

In [ ]:
WITH 
cancel_orders AS (
    SELECT 
        order_id
    FROM 
        user_actions
    WHERE
        action = 'cancel_order'
),
first_order AS (
    SELECT
        DATE(MIN(time)) AS date,
        user_id
    FROM 
        user_actions
    WHERE
        order_id NOT IN (SELECT order_id FROM cancel_orders)
    GROUP BY
        user_id
)
SELECT
    date,
    COUNT(user_id) AS first_orders
FROM 
    first_order
GROUP BY
    date
ORDER BY
    date

Выберите все колонки из таблицы orders и дополнительно в качестве последней колонки укажите функцию unnest, применённую к колонке product_ids. Эту последнюю колонку назовите product_id. Больше ничего с данными делать не нужно.
Добавьте в запрос оператор LIMIT и выведите только первые 100 записей результирующей таблицы.

In [ ]:
SELECT 
    order_id, 
    creation_time, 
    product_ids,
    unnest(product_ids) AS product_id
FROM
    orders
LIMIT 
    100

Используя функцию unnest, определите 10 самых популярных товаров в таблице orders.

In [ ]:
WITH 
cancel_order AS (
    SELECT 
        order_id
    FROM 
        user_actions
    WHERE
        action = 'cancel_order'
),
product_purchases AS (
    SELECT 
        unnest(product_ids) AS product_id,
        COUNT(*) AS times_purchased
    FROM 
        orders
    WHERE
        order_id NOT IN (SELECT order_id FROM cancel_order)
    GROUP BY
        product_id
    ORDER BY 
        times_purchased DESC
    LIMIT
        10
)
SELECT 
    product_id, 
    times_purchased
FROM
    product_purchases
ORDER BY 
    product_id

Из таблицы orders выведите id и содержимое заказов, которые включают хотя бы один из пяти самых дорогих товаров, доступных в нашем сервисе.

In [ ]:
WITH
top_5_price AS (
    SELECT 
        product_id
    FROM 
        products
    ORDER BY 
        price DESC
    LIMIT 
        5
),
products_list AS (
    SELECT 
        order_id,
        product_ids, 
        unnest(product_ids) AS product_id
    FROM
    orders    
)
SELECT
    DISTINCT(order_id),
    product_ids
FROM
    products_list
WHERE 
    product_id IN (SELECT product_id FROM top_5_price)
ORDER BY 
    order_id
    